In [3]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt

In [7]:
# import platform

# from matplotlib import font_manager, rc
# plt.rcParams['axes.unicode_minus'] = False

# if platform.system() == 'Darwin':  # 맥OS 
#     rc('font', family='AppleGothic')
# elif platform.system() == 'Windows':  # 윈도우
#     path = "c:/Windows/Fonts/malgun.ttf"
#     font_name = font_manager.FontProperties(fname=path).get_name()
#     rc('font', family=font_name)
# else:
#     rc('font', family='NanumGothic')

In [4]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
JDBC = {
    'url':'jdbc:oracle:thin:@realestate_high?TNS_ADMIN=/home/big/study/db/Wallet_REALESTATE'
    ,'props':{
        'user':'dw_realestate',
        'password':'123qwe!@#QWE'
    }   
}

### 전국소유유형별 등기 구분 수
CLS 소유유형 ->owner_cls

TOT 건수 ->소유유형별로 그룹바이 후 건수 count

RATE 비율->

In [6]:
realestate_own = spark.read.jdbc(url=JDBC['url'], table='REALESTATE_OWN', properties=JDBC['props'])
df_loc = spark.read.jdbc(url=JDBC['url'], table='LOC', properties=JDBC['props'])

In [7]:
# df_real=realestate_own.select(col('PROPERTY_TYPE'),col('TOT') / sum_value * 100) \
# .groupBy(col('PROPERTY_TYPE')) \
# .agg(count('PROPERTY_TYPE').alias('TOT'))
# df_real.show()

df_loc.createOrReplaceTempView('LOC')

In [8]:
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
a=spark.sql('''
select owner_cls as cls,count(owner_cls)as tot
,round((count(owner_cls)/(select count(owner_cls) from realestate_own)*100),2) as rate
from realestate_own
group by owner_cls;
''')
a.show()

+------------------------+-----+-----+
|                     cls|  tot| rate|
+------------------------+-----+-----+
|                  외국인|  823| 5.83|
|                    기타|    2| 0.01|
|                  비법인|  430| 3.05|
|                  내국인|11284|79.99|
|                재외국민|   62| 0.44|
|                    법인|  997| 7.07|
|국가기관 및 지방자치단체|  508|  3.6|
+------------------------+-----+-----+



### 서울시 소유유형별 등기 구분 수


In [12]:
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
df_loc.createOrReplaceTempView('LOC')
a=spark.sql('''
select owner_cls as cls,count(owner_cls)as tot
,round((count(owner_cls)/(select count(owner_cls) from realestate_own)*100),2) as rate
from realestate_own ro
inner join loc l
on l.loc_code=ro.regn_code
where sido='서울특별시'
group by owner_cls;
''')
a.show()

+------------------------+----+----+
|                     cls| tot|rate|
+------------------------+----+----+
|                  외국인| 153|1.08|
|                    기타|   1|0.01|
|                  비법인|  31|0.22|
|                  내국인|1148|8.14|
|                재외국민|  10|0.07|
|                    법인| 117|0.83|
|국가기관 및 지방자치단체|  33|0.23|
+------------------------+----+----+



In [ ]:
#나이대
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
a=spark.sql('''
select OWNER_AGES as ages,count(OWNER_AGES)as tot
,round((count(OWNER_AGES)/(select count(OWNER_AGES) from realestate_own)*100),2) as rate
from realestate_own
group by OWNER_AGES;
''')
a.show()

In [13]:
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
a=spark.sql('''
select OWNER_AGES as ages,count(OWNER_AGES)as tot
,round((count(OWNER_AGES)/(select count(OWNER_AGES) from realestate_own)*100),2) as rate
from realestate_own ro
inner join loc l
on l.loc_code=ro.regn_code
where sido='서울특별시'
group by OWNER_AGES;
''')
a.show()

+----------------+---+----+
|            ages|tot|rate|
+----------------+---+----+
|       30세~39세|211| 1.5|
|            기타|182|1.29|
|0~18세(미성년자)| 62|0.44|
|       50세~59세|244|1.73|
|       40세~49세|219|1.55|
|       60세~69세|233|1.65|
|       70세 이상|191|1.35|
|       19세~29세|151|1.07|
+----------------+---+----+



In [ ]:
#성별
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
a=spark.sql('''
select OWNER_SEX as sex,count(OWNER_SEX)as tot
,round((count(OWNER_SEX)/(select count(OWNER_SEX) from realestate_own)*100),2) as rate
from realestate_own
group by OWNER_SEX;
''')
a.show()

In [14]:
realestate_own.createOrReplaceTempView("REALESTATE_OWN")
a=spark.sql('''
select OWNER_SEX as sex,count(OWNER_SEX)as tot
,round((count(OWNER_SEX)/(select count(OWNER_SEX) from realestate_own)*100),2) as rate
from realestate_own ro
inner join loc l
on l.loc_code=ro.regn_code
where sido='서울특별시'
group by OWNER_SEX;
''')
a.show()

+----+---+----+
| sex|tot|rate|
+----+---+----+
|기타|182|1.29|
|여자|640|4.54|
|남자|671|4.76|
+----+---+----+

